# Uso de OBJDUMP para inspección de binarios

In [1]:
! lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

Archivo de código C en el que suman los elementos de dos vectores de enteros, y el resultado se almacena en otro vector

In [2]:
%%file vec_sum.c

void vec_sum(int* a, int* b, int* c, int N){
    for(int i = 0; i < N; i++){
        c[i] = a[i] + b[i];
    }
}

Overwriting vec_sum.c


Se compila con las siguientes opciones:



*   `-Os` para que ocupe menos espaico
*   `-c` para que cree un `object file`
*   `-o` para indicar el archivo de salida



In [3]:
! gcc -Os -c vec_sum.c -o vec_sum.o

Usar `objdump` para ver las instrucciones del `object file` generado con las siguientes opciones:



*   `-M intel` para que use sintaxis de intel
*   `-j .text` para que solo muestre el segmento de código
*   `-D` para que muestre el disassembly



In [4]:
! objdump -M intel -j .text -D vec_sum.o


vec_sum.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <vec_sum>:
   0:	f3 0f 1e fa          	endbr64 
   4:	31 c0                	xor    eax,eax
   6:	39 c1                	cmp    ecx,eax
   8:	7e 11                	jle    1b <vec_sum+0x1b>
   a:	44 8b 04 86          	mov    r8d,DWORD PTR [rsi+rax*4]
   e:	44 03 04 87          	add    r8d,DWORD PTR [rdi+rax*4]
  12:	44 89 04 82          	mov    DWORD PTR [rdx+rax*4],r8d
  16:	48 ff c0             	inc    rax
  19:	eb eb                	jmp    6 <vec_sum+0x6>
  1b:	c3                   	ret    


Ahora usted analice la salida de `objdump` y cuente cuantas instrucciones ha ejecutado este código. Tenga en cuenta que la cantidad de instrucciones ejecutadas cambian con el valor de `N`.

Luego de hacer una inspección para unos valores pequeños de N, se llega a la siguiente tabla.

| N | #instrucciones |
|:-:|:--------------:|
| 0 |        5       |
| 1 |       12       |
| 2 |       19       |
| 3 |       26       |

Se podría decir que, en general, para este código la cantidad de instrucciones es `#instrucciones = 7 * N + 5`. Con esto, ya es posible calcular el CPI asociado a un tamaño `N`.

In [5]:
%%file test.c

#include <x86intrin.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

int* crear_arr(int N){
    int* v = (int*)malloc(sizeof(int)*N);
    for(int i = 0; i < N; i++){
        v[i] = rand() % 9;
    }
    return v;    
}

void fill_arr(int* v, int N){
    for(int i = 0; i < N; i++){
        v[i] = rand() % 9;
    }
}


void vec_sum(int* a, int* b, int* c, int N){
    for(int i = 0; i < N; i++){
        c[i] = a[i] + b[i];
    }
}

int calc_num_instr(int N){
    return 7*N+5;
}

double calcular_CPI(long int num_cic, int num_inst){
    return (double)num_cic / (double)num_inst;
}

int main(){
    
    srand(time(NULL));

    int N = 8;

    int* a = crear_arr(N);
    int* b = crear_arr(N);
    int* c = crear_arr(N);

    long int tic, toc, ciclos;

    tic = __rdtsc();
    vec_sum(a, b, c, N);
    toc = __rdtsc();

    ciclos = toc - tic;

    int num_instr = calc_num_instr(N);

    double CPI = calcular_CPI(ciclos, num_instr);

    printf("#instrucciones:%d\n", num_instr);
    printf("ciclos:%ld\n", ciclos);
    printf("CPI:%lf\n",CPI);

    return 0;
}



Overwriting test.c


In [6]:
! gcc test.c -o test

In [1]:
! ./test

#instrucciones:61
ciclos:588
CPI:9.639344


Note que esto es una aproximación muy ingenua, porque para medir el CPI deberíamos probar la función más de una vez y realizar un cálculo promedio.